Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model with ONNX Runtime on GPU

In this tutorial, you'll learn how to load a Bert model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime and NVIDIA GPU. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text from the corresponding reading passage, or the question might be unanswerable.

This notebook is for GPU inference. For CPU inference, please look at another notebook [Inference PyTorch Bert Model with ONNX Runtime on CPU](PyTorch_Bert-Squad_OnnxRuntime_CPU.ipynb).

## 0. Prerequisites ##
It requires your machine to have a GPU, and a python environment with [PyTorch](https://pytorch.org/) installed before running this notebook.

#### GPU Environment Setup using AnaConda

First, we install [AnaConda](https://www.anaconda.com/distribution/) in a target machine and open an AnaConda prompt window when it is done. Then run the following commands to create a conda environment. This notebook is tested with PyTorch 1.5.0 and OnnxRuntime 1.3.0.

```console
conda create -n gpu_env python=3.6
conda activate gpu_env
conda install -c anaconda ipykernel
conda install -c conda-forge ipywidgets
python -m ipykernel install --user --name=gpu_env
jupyter notebook
```
Finally, launch Jupyter Notebook and you can choose gpu_env as kernel to run this notebook.

Onnxruntime-gpu need specified version of CUDA and cuDNN. You can find the Requirements [here]( http://www.onnxruntime.ai/docs/how-to/install.html). Remember to add the directories to PATH environment variable (See [CUDA and cuDNN Path](#CUDA-and-cuDNN-Path) below).

In [1]:
import sys

run_install = False # Only need install once
if run_install:
    if sys.platform in ['linux', 'win32']: # Linux or Windows
        !{sys.executable} -m pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
    else: # Mac
        print("PyTorch 1.9 MacOS Binaries do not support CUDA, install from source instead")

    !{sys.executable} -m pip install onnxruntime-gpu==1.8.1 onnx==1.9.0 onnxconverter_common==1.8.1

    # Install other packages used in this notebook.
    !{sys.executable} -m pip install transformers==4.8.2
    !{sys.executable} -m pip install psutil pytz pandas py-cpuinfo py3nvml coloredlogs wget netron sympy

In [2]:
!pip uninstall -y onnx onnxruntime onnxruntime-gpu
!pip install onnx onnxruntime-gpu

Found existing installation: onnx 1.12.0
Uninstalling onnx-1.12.0:
  Successfully uninstalled onnx-1.12.0
Found existing installation: onnxruntime-gpu 1.13.1
Uninstalling onnxruntime-gpu-1.13.1:
  Successfully uninstalled onnxruntime-gpu-1.13.1
Defaulting to user installation because normal site-packages is not writeable
  Using cached onnx-1.12.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached onnxruntime_gpu-1.13.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (115.3 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import torch
import onnx
import onnxruntime
import transformers
print("pytorch:", torch.__version__)
#print("onnxruntime:", onnxruntime.__version__)
#print("onnx:", onnx.__version__)
print("transformers:", transformers.__version__)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


pytorch: 1.13.0+cu116
transformers: 4.24.0


## 1. Load Pretrained Bert model ##

We begin by downloading the SQuAD data file and store them in the specified location. 

In [4]:
!pip install wget

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=725d99feaa84b4ef7337637704f1a92c2e424f0f07d97f591e64f65b5c76dcb2
  Stored in directory: /var/snell_home/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import os

cache_dir = "./squad"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")

Let's first define some constant variables.

In [6]:
# Whether allow overwriting existing ONNX model and download the latest script from GitHub
enable_overwrite = True

# Total samples to inference, so that we can get average latency
total_samples = 1000

# ONNX opset version
opset_version=11

Specify some model configuration variables.

In [7]:
# fine-tuned model from https://huggingface.co/models?search=squad
model_name_or_path = "bert-large-uncased-whole-word-masking-finetuned-squad"
max_seq_length = 128
doc_stride = 128
max_query_length = 64

Start to load model from pretrained. This step could take a few minutes. 

In [8]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
# load some examples
from transformers.data.processors.squad import SquadV1Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features( 
            examples=examples[:total_samples], # convert enough examples for this notebook
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

add example index and unique id: 100%|██████████| 1000/1000 [00:00<00:00, 255376.52it/s]


## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [9]:
output_dir = "./onnx"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)   
export_model_path = os.path.join(output_dir, 'bert-base-cased-squad_opset{}.onnx'.format(opset_version))

import torch
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

# Get the first example data to run the model and export it to ONNX
data = dataset[0]
inputs = {
    'input_ids':      data[0].to(device).reshape(1, max_seq_length),
    'attention_mask': data[1].to(device).reshape(1, max_seq_length),
    'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
}

# Set model to inference mode, which is required before exporting the model because some operators behave differently in 
# inference and training mode.
model.eval()
model.to(device)

if enable_overwrite or not os.path.exists(export_model_path):
    with torch.no_grad():
        symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
        torch.onnx.export(model,                                            # model being run
                          args=tuple(inputs.values()),                      # model input (or a tuple for multiple inputs)
                          f=export_model_path,                              # where to save the model (can be a file or file-like object)
                          opset_version=opset_version,                      # the ONNX version to export the model to
                          do_constant_folding=True,                         # whether to execute constant folding for optimization
                          input_names=['input_ids',                         # the model's input names
                                       'input_mask', 
                                       'segment_ids'],
                          output_names=['start', 'end'],                    # the model's output names
                          dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                        'input_mask' : symbolic_names,
                                        'segment_ids' : symbolic_names,
                                        'start' : symbolic_names,
                                        'end' : symbolic_names})
        print("Model exported at ", export_model_path)

Model exported at  ./onnx/bert-base-cased-squad_opset11.onnx


## 3. PyTorch Inference ##
Use PyTorch to evaluate an example input for comparison purpose.

In [10]:
import time

# Measure the latency. It is not accurate using Jupyter Notebook, it is recommended to use standalone python script.
latency = []
with torch.no_grad():
    for i in range(total_samples):
        data = dataset[i]
        inputs = {
            'input_ids':      data[0].to(device).reshape(1, max_seq_length),
            'attention_mask': data[1].to(device).reshape(1, max_seq_length),
            'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
        }
        start = time.time()
        outputs = model(**inputs)
        latency.append(time.time() - start)
print("PyTorch {} Inference time = {} ms".format(device.type, format(sum(latency) * 1000 / len(latency), '.2f')))

PyTorch cuda Inference time = 12.19 ms


## 4. Inference ONNX Model with ONNX Runtime ##

### CUDA and cuDNN Path
onnxruntime-gpu has dependency on [CUDA](https://developer.nvidia.com/cuda-downloads) and [cuDNN](https://developer.nvidia.com/cudnn). Required CUDA version can be found [here](http://www.onnxruntime.ai/docs/reference/execution-providers/CUDA-ExecutionProvider.html#requirements)


In [11]:
# Change to True when onnxruntime (like onnxruntime-gpu 1.0.0 ~ 1.1.2) cannot be imported.
add_cuda_path = False

# For Linux, see https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html#environment-setup
# Below is example for Windows
if add_cuda_path:
    cuda_dir = 'D:/NVidia/CUDA/v11.0/bin'
    cudnn_dir = 'D:/NVidia/CUDA/v11.0/bin'
    if not (os.path.exists(cuda_dir) and os.path.exists(cudnn_dir)):
        raise ValueError("Please specify correct path for CUDA and cuDNN. Otherwise onnxruntime cannot be imported.")
    else:
        if cuda_dir == cudnn_dir:
            os.environ["PATH"] = cuda_dir + ';' + os.environ["PATH"]
        else:
            os.environ["PATH"] = cuda_dir + ';' + cudnn_dir + ';' + os.environ["PATH"]

Now we are ready to inference the model with ONNX Runtime.

In [12]:
import psutil
import onnxruntime
import numpy

assert 'CUDAExecutionProvider' in onnxruntime.get_available_providers()
device_name = 'gpu'

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
# Note that this will increase session creation time so enable it for debugging only.
sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_{}.onnx".format(device_name))

# Please change the value according to best setting in Performance Test Tool result.
sess_options.intra_op_num_threads=psutil.cpu_count(logical=True)

session = onnxruntime.InferenceSession(export_model_path, sess_options, providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

latency = []
for i in range(total_samples):
    data = dataset[i]
    ort_inputs = {
        'input_ids':  data[0].cpu().reshape(1, max_seq_length).numpy(),
        'input_mask': data[1].cpu().reshape(1, max_seq_length).numpy(),
        'segment_ids': data[2].cpu().reshape(1, max_seq_length).numpy()
    }
    start = time.time()
    ort_outputs = session.run(None, ort_inputs)
    latency.append(time.time() - start)
    
print("OnnxRuntime {} Inference time = {} ms".format(device_name, format(sum(latency) * 1000 / len(latency), '.2f')))

OnnxRuntime gpu Inference time = 5.73 ms


We can compare the output of PyTorch and ONNX Runtime. We can see some results are not close. It is because ONNX Runtime uses some approximation in CUDA optimization. Based on our evaluation on SQuAD data set, F1 score is on par for models before and after optimization.

In [13]:
print("***** Verifying correctness *****")
for i in range(2):    
    print('PyTorch and ONNX Runtime output {} are close:'.format(i), numpy.allclose(ort_outputs[i], outputs[i].cpu(), rtol=1e-02, atol=1e-02))
    diff = ort_outputs[i] - outputs[i].cpu().numpy()
    max_diff = numpy.max(numpy.abs(diff))
    avg_diff = numpy.average(numpy.abs(diff))
    print(f'maximum_diff={max_diff} average_diff={avg_diff}')

***** Verifying correctness *****
PyTorch and ONNX Runtime output 0 are close: True
maximum_diff=0.003519296646118164 average_diff=0.000500253401696682
PyTorch and ONNX Runtime output 1 are close: True
maximum_diff=0.0038802623748779297 average_diff=0.0004846229567192495


### Inference with Actual Sequence Length
Note that ONNX model is exported using dynamic length axis. It is recommended to use actual sequence input without padding instead of fixed length input for best performance. Let's see how it can be applied to this model.

From an example input below, we can see zero padding at the end of each sequence.

In [14]:
# An example input (we can see padding). From attention_mask, we can deduce the actual length.
inputs

{'input_ids': tensor([[  101,  2054,  2329,  2694,  2897,  2097,  4287,  1996,  3565,  4605,
           1029,   102,  1999,  1996,  2142,  2983,  1010,  4035,  2557,  1019,
           2444,  1998,  1019,  2444,  2998,  4469,  2097,  4287,  1996,  5049,
           1012,  1996,  4035,  2097,  4287,  2049,  2219,  2329,  2394,  3743,
           1010,  2007,  6754, 10184,  1010, 12270, 10589,  1998,  6857,  8945,
          18505,  2006,  8570,  1012,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

The original sequence length is 128. After removing paddings, the sequence length is reduced. Input with smaller sequence length need less computation, thus we can see there is improvement on inference latency. 

In [15]:
import statistics

latency = []
lengths = []
for i in range(total_samples):
    data = dataset[i]
    # Instead of using fixed length (128), we can use actual sequence length (less than 128), which helps to get better performance.
    actual_sequence_length = sum(data[1].numpy())
    lengths.append(actual_sequence_length)
    opt_inputs = {
        'input_ids':  data[0].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length),
        'input_mask': data[1].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length),
        'segment_ids': data[2].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length)
    }
    start = time.time()
    opt_outputs = session.run(None, opt_inputs)
    latency.append(time.time() - start)
print("Average length", statistics.mean(lengths))
print("OnnxRuntime {} Inference time with actual sequence length = {} ms".format(device_name, format(sum(latency) * 1000 / len(latency), '.2f')))

Average length 94
OnnxRuntime gpu Inference time with actual sequence length = 5.36 ms


Let's compare the output and see whether the results are close.

**Note**: Need end-to-end evaluation on performance and accuracy if you use this strategy.

In [16]:
print("***** Comparing results with/without paddings *****")
for i in range(2):
    print('Output {} are close:'.format(i), numpy.allclose(opt_outputs[i], ort_outputs[i][:,:len(opt_outputs[i][0])], rtol=1e-03, atol=1e-03))

***** Comparing results with/without paddings *****
Output 0 are close: True
Output 1 are close: True


## 5. Offline Optimization and Test Tools

It is recommended to try [OnnxRuntime Transformer Model Optimization Tool](https://github.com/microsoft/onnxruntime/blob/main/onnxruntime/python/tools/transformers) on the exported ONNX models. It could help verify whether the model can be fully optimized, and get performance test results.

#### Transformer Optimizer

Although OnnxRuntime could optimize Bert model exported by PyTorch. Sometime, model cannot be fully optimized due to different reasons:
* A new subgraph pattern is generated by new version of export tool, and the pattern is not covered by older version of OnnxRuntime. 
* The exported model uses dynamic axis and this makes it harder for shape inference of the graph. That blocks some optimization to be applied.
* Some optimization is better to be done offline. Like change input tensor type from int64 to int32 to avoid extra Cast nodes, or convert model to float16 to achieve better performance in V100 or T4 GPU.

We have python script **optimizer.py**, which is more flexible in graph pattern matching and model conversion (like float32 to float16). You can also use it to verify whether a Bert model is fully optimized.

In this example, we can see that it introduces optimization that is not provided by onnxruntime: SkipLayerNormalization and bias fusion, which is not fused in OnnxRuntime due to shape inference as mentioned.

It will also tell whether the model is fully optimized or not. If not, that means you might need change the script to fuse some new pattern of subgraph.

Example Usage:
```
from onnxruntime.transformers import optimizer
optimized_model = optimizer.optimize_model(export_model_path, model_type='bert', num_heads=12, hidden_size=768)
optimized_model.save_model_to_file(optimized_model_path)
```

You can also use command line like the following:

#### Float32 Model
Let us optimize the ONNX model using the script. The first example will output model with float32 to store weights. This is the choice for most GPUs without Tensor Core.

If your GPU (like V100 or T4) has Tensor Core, jump to [Float16 Model](#6.-Model-Optimization-with-Float16) section since that will give you better performance than Float32 model.

In [17]:
optimized_fp32_model_path = './onnx/bert-base-cased-squad_opt_{}_fp32.onnx'.format('gpu' if use_gpu else 'cpu')

!{sys.executable} -m onnxruntime.transformers.optimizer --input $export_model_path --output $optimized_fp32_model_path

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
               apply: Fused LayerNormalization count: 49
               apply: Fused Gelu count: 24
               apply: Fused SkipLayerNormalization count: 48
               apply: Fused Attention count: 24
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 5 nodes are removed
               apply: Fused EmbedLayerNormalization(with mask) count: 1
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 10 nodes are removed
               apply: Fused BiasGelu count: 24
               apply: Fused SkipLayerNormalization(add bias) count: 48
            optimize: opset version: 11
  save_model_to_file: Sort graphs in topological order
  save_model_to_file: Model saved to ./onnx/bert-base-cased-squad_opt_gpu_fp32.onnx
get_fused_operator_statistics: O

#### Optimized Graph
We can open the optimized model using [Netron](https://github.com/lutzroeder/netron) to visualize.

The graph is like the following:
<img src='images/optimized_bert_gpu.png'>

Sometime, optimized graph is slightly different. For example, FastGelu is replaced by BiasGelu for CPU inference; When the option --input_int32 is used, Cast nodes for inputs are removed.

In [18]:
!pip install netron

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:0000:010:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [19]:
import netron

# change it to True if want to view the optimized model in browser
enable_netron = True
if enable_netron:
    # If you encounter error "access a socket in a way forbidden by its access permissions", install Netron as standalone application instead.
    netron.start(optimized_fp32_model_path)

Serving './onnx/bert-base-cased-squad_opt_gpu_fp32.onnx' at http://localhost:8080


### Performance Test Tool

The following will create 1000 random inputs of batch_size 1 and sequence length 128, then measure the average latency and throughput numbers.

Note that the test uses fixed sequence length. If you use [dynamic sequence length](#Inference-with-Actual-Sequence-Length), actual performance depends on the distribution of sequence length.

**Attention**: Latency numbers from Jupyter Notebook are not accurate. See [Attional Info](#7.-Additional-Info) for more info.

In [20]:
GPU_OPTION = '--use_gpu' if use_gpu else ''

!{sys.executable} -m onnxruntime.transformers.bert_perf_test --model $optimized_fp32_model_path --batch_size 1 --sequence_length 128 --samples 1000 --test_times 1 $GPU_OPTION

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
test setting TestSetting(batch_size=1, sequence_length=128, test_cases=1000, test_times=1, use_gpu=True, use_io_binding=False, provider=None, intra_op_num_threads=None, seed=3, verbose=False, log_severity=2)
Generating 1000 samples for batch_size=1 sequence_length=128
Running test: model=bert-base-cased-squad_opt_gpu_fp32.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=20,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,use_gpu=True
Average latency = 4.65 ms, Throughput = 215.23 QPS
Running test: model=bert-base-cased-squad_opt_gpu_fp32.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=15,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,use_gpu=True
Average latency = 4.67 ms, Throughput = 214.05 QPS
Running te

Let's load the summary file and take a look. Note that blank value in OMP_NUM_THREADS or OMP_WAIT_POLICY means the environment variable does not exist.

In [21]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_GPU_B1_S128_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file)
print("Float32 model perf results from", latest_result_file)
# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu']
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float32 model perf results from ./onnx/perf_results_GPU_B1_S128_20221117-182747.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),intra_op_num_threads
0,4.65,4.65,4.65,4.66,4.67,4.68,215.23,20
1,4.67,4.67,4.68,4.69,4.70,4.71,214.05,15
2,4.69,4.69,4.69,4.70,4.71,4.73,213.44,14
3,4.70,4.70,4.72,4.73,4.73,4.75,212.55,13
4,4.71,4.71,4.72,4.73,4.74,4.81,212.30,11
5,4.71,4.71,4.72,4.73,4.74,4.76,212.23,10
6,4.72,4.72,4.73,4.75,4.75,4.77,211.76,9
7,4.72,4.72,4.73,4.74,4.75,4.77,211.75,12
8,4.73,4.72,4.74,4.75,4.76,4.81,211.61,3
9,4.73,4.73,4.74,4.76,4.76,4.79,211.28,8


From above result, we can see that latency is very close for different settings. The default setting (intra_op_num_threads=0, OMP_NUM_THREADS and OMP_WAIT_POLICY does not exist) performs the best. 

### Model Results Comparison Tool

When a BERT model is optimized, some approximation is used in calculation. If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare the inference outputs of the original and optimized models. If outputs are all close, it is safe to use the optimized model.

For GPU inference, the absolute or relative difference is larger than those numbers of CPU inference. Note that slight difference in output will not impact final result. We did end-to-end evaluation using SQuAD data set using a fine-tuned squad model, and F1 score is almost the same before/after optimization.

In [22]:
!{sys.executable} -m onnxruntime.transformers.compare_bert_results --baseline_model $export_model_path --optimized_model $optimized_fp32_model_path --batch_size 1 --sequence_length 128 --samples 100 --rtol 0.01 --atol 0.01 $GPU_OPTION

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
100% passed for 100 random inputs given thresholds (rtol=0.01, atol=0.01).
maximum absolute difference=0.029673099517822266
maximum relative difference=0.18719154596328735


## 6. Model Optimization with Float16

The optimizer.py script have an option **--float16** to convert model to use float16 to store weights. After the conversion, it could be faster to run in GPU with tensor cores like V100 or T4.

Let's run tools to measure the performance on V100. The results show significant performance improvement: latency is about 3.4 ms for float32 model, and 1.8 ms for float16 model.

In [23]:
optimized_fp16_model_path = './onnx/bert-base-cased-squad_opt_{}_fp16.onnx'.format('gpu' if use_gpu else 'cpu')
!{sys.executable} -m onnxruntime.transformers.optimizer --input $export_model_path --output $optimized_fp16_model_path --float16

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
               apply: Fused LayerNormalization count: 49
               apply: Fused Gelu count: 24
               apply: Fused SkipLayerNormalization count: 48
               apply: Fused Attention count: 24
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 5 nodes are removed
               apply: Fused EmbedLayerNormalization(with mask) count: 1
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 10 nodes are removed
               apply: Fused BiasGelu count: 24
               apply: Fused SkipLayerNormalization(add bias) count: 48
            optimize: opset version: 11
  save_model_to_file: Sort graphs in topological order
  save_model_to_file: Model saved to ./onnx/bert-base-cased-squad_opt_gpu_fp16.onnx
get_fused_operator_statistics: O

In [24]:
GPU_OPTION = '--use_gpu' if use_gpu else ''
!python -m onnxruntime.transformers.bert_perf_test --model $optimized_fp16_model_path --batch_size 1 --sequence_length 128 --samples 1000 --test_times 1 $GPU_OPTION

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
test setting TestSetting(batch_size=1, sequence_length=128, test_cases=1000, test_times=1, use_gpu=True, use_io_binding=False, provider=None, intra_op_num_threads=None, seed=3, verbose=False, log_severity=2)
Generating 1000 samples for batch_size=1 sequence_length=128
Running test: model=bert-base-cased-squad_opt_gpu_fp16.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=20,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,use_gpu=True
Average latency = 2.45 ms, Throughput = 408.91 QPS
Running test: model=bert-base-cased-squad_opt_gpu_fp16.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=15,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,use_gpu=True
Average latency = 2.44 ms, Throughput = 409.55 QPS
Running te

In [25]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_GPU_B1_S128_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file)
print("Float32 model perf results from", latest_result_file)
# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu']
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float32 model perf results from ./onnx/perf_results_GPU_B1_S128_20221117-183252.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),intra_op_num_threads
0,2.44,2.44,2.45,2.46,2.46,2.48,409.55,15
1,2.45,2.44,2.45,2.46,2.46,2.47,408.91,20
2,2.46,2.45,2.46,2.47,2.48,2.52,407.19,13
3,2.46,2.46,2.46,2.47,2.48,2.49,406.86,14
4,2.47,2.47,2.48,2.49,2.51,2.54,404.40,12
5,2.47,2.47,2.48,2.49,2.49,2.52,404.30,10
6,2.48,2.47,2.48,2.49,2.52,2.58,403.62,11
7,2.48,2.48,2.48,2.49,2.50,2.51,403.53,9
8,2.48,2.48,2.49,2.50,2.50,2.52,402.74,8
9,2.49,2.49,2.50,2.50,2.51,2.53,402.09,3


### Throughput Tuning

Some application need best throughput under some constraint on latency. This can be done by testing performance of different batch sizes. The tool could help on this.

Here is an example that check the performance of multiple batch sizes (1, 2, 4, 8, 16, 32 and 64) using default settings.

In [26]:
GPU_OPTION = '--use_gpu' if use_gpu else ''
THREAD_SETTING = '--intra_op_num_threads 3'
!{sys.executable} -m onnxruntime.transformers.bert_perf_test --model $optimized_fp16_model_path --batch_size 1 2 4 8 16 32 --sequence_length 128 --samples 1000 --test_times 1 $THREAD_SETTING $GPU_OPTION

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
test setting TestSetting(batch_size=32, sequence_length=128, test_cases=1000, test_times=1, use_gpu=True, use_io_binding=False, provider=None, intra_op_num_threads=3, seed=3, verbose=False, log_severity=2)
Generating 1000 samples for batch_size=32 sequence_length=128
Running test: model=bert-base-cased-squad_opt_gpu_fp16.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=3,batch_size=32,sequence_length=128,test_cases=1000,test_times=1,use_gpu=True
Average latency = 39.44 ms, Throughput = 811.26 QPS
test setting TestSetting(batch_size=1, sequence_length=128, test_cases=1000, test_times=1, use_gpu=True, use_io_binding=False, provider=None, intra_op_num_threads=3, seed=3, verbose=False, log_severity=2)
Generating 1000 samples for batch_size=1 sequence_l

In [30]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file)
print("Float16 model summary from", latest_result_file)
columns_to_remove = ['model', 'graph_optimization_level', 'test_cases', 'test_times', 'use_gpu', 'sequence_length']
columns_to_remove.extend(['intra_op_num_threads'])
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data['Actual Latency(ms)'] = result_data['Latency(ms)'] / result_data['batch_size']
result_data

Float16 model summary from ./onnx/perf_results_GPU_B1-2-4-8-16-32_S128_20221117-183449.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),batch_size,Actual Latency(ms)
0,2.52,2.52,2.53,2.54,2.56,2.62,396.52,1,2.520000
1,3.77,3.77,3.78,3.81,3.81,3.92,530.25,2,1.885000
2,5.95,5.95,5.99,6.01,6.02,6.13,672.09,4,1.487500
3,10.77,10.78,10.79,10.87,10.88,10.90,743.02,8,1.346250
4,20.03,20.03,20.05,20.19,20.20,20.29,798.81,16,1.251875
5,39.44,39.53,39.84,39.86,39.87,40.00,811.26,32,1.232500


## 7. Additional Info

Note that running Jupyter Notebook has significant impact on performance result. You can close Jupyter Notebook and other applications, then run the performance test in a console to get more accurate performance numbers.

We have a [benchmark script](https://github.com/microsoft/onnxruntime/blob/main/onnxruntime/python/tools/transformers/run_benchmark.sh). It is recommended to use it measure inference speed of OnnxRuntime.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/main/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. You might get slower or faster result according to your hardware.

In [28]:
!pip install py-cpuinfo
!pip install py3nvml

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.1 MB/s eta 0:00:00
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [29]:
!{sys.executable} -m onnxruntime.transformers.machine_info --silent

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
{
  "gpu": {
    "driver_version": "510.47.03",
    "devices": [
      {
        "memory_total": 51527024640,
        "memory_available": 44790775808,
        "name": "NVIDIA RTX A6000"
      },
      {
        "memory_total": 12884901888,
        "memory_available": 12642877440,
        "name": "NVIDIA TITAN V"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Xeon(R) W-2155 CPU @ 3.30GHz",
    "cores": 10,
    "logical_cores": 20,
    "hz": "1201470000,0",
    "l2_cache": 10485760,
    "flags": "3dnowprefetch,abm,acpi,adx,aes,aperfmperf,apic,arat,arch_capabilities,arch_perfmon,art,avx,avx2,avx512bw,avx512cd,avx512dq,avx512f,avx512vl,bmi1,bmi2,bts,cat_l3,cdp_l3,clflush,clflushopt,clwb,cmov,constant_tsc,cpuid,cpuid_fault,cqm,cqm_llc,cqm_mbm_local,cqm_mbm_tot